<a href="https://colab.research.google.com/github/Jibrangit/DL_Assignments/blob/HW6/HW6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, UpSampling2D, ZeroPadding2D

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import matplotlib.pyplot as plt
from cv2 import resize
import random

def random_rearrange (X_tr, y_tr, seed):
    np.random.seed(seed)
    np.random.shuffle(X_tr)
    np.random.seed(seed)
    np.random.shuffle(y_tr)

def rechannel(x):
  rechanelled_img = np.reshape(np.repeat(x, 3, axis=1), [48, 48, 3])
  return rechanelled_img
  
y_load = np.load('/content/drive/MyDrive/Colab Notebooks/HW6/ages.npy')
x_loaded = np.load('/content/drive/MyDrive/Colab Notebooks/HW6/faces.npy')
dataset_size = len(x_loaded)

x_load = np.zeros([dataset_size, 48, 48, 3])
for i in range(0, dataset_size):
  x_load[i, :, :, :] = rechannel(x_loaded[i, :, :])

no_data = int(x_load.shape[0])
train_size = int(0.8*0.8*no_data)
test_size = int(0.2*no_data)
print(y_load[0])   
random_rearrange(x_load, y_load, 0)
print(y_load[0])                        #Just checking if shuffling ahs been done coorrectly.
(x_test, x_train, x_valid) = x_load[:test_size], x_load[test_size:test_size+train_size],  x_load[test_size+train_size:]
(y_test, y_train, y_valid) = y_load[:test_size], y_load[test_size:test_size+train_size], y_load[test_size+train_size:]
x_train = x_train.astype('float32') / 255
x_valid = x_valid.astype('float32') / 255
x_test = x_test.astype('float32') / 255

40.0
48.0


In [4]:
model = Sequential()

In [5]:
from keras.applications import VGG16

model.add(ZeroPadding2D(4, input_shape=(48, 48, 3)))
model.add(UpSampling2D(size=(4, 4)))
model.build()

model.add(VGG16(weights='imagenet', include_top=False, input_shape=[224, 224, 3]))

model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=1, activation="relu")) 

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d (ZeroPadding2 (None, 56, 56, 3)         0         
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 224, 224, 3)       0         
_________________________________________________________________
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 4096)              102764544 
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 4

In [7]:
import keras.backend as K
from keras.metrics import RootMeanSquaredError
from keras.optimizers import Adam
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

opt = Adam(lr=0.0001)
model.compile(
  loss=root_mean_squared_error,
  optimizer=opt,
  metrics=[RootMeanSquaredError()]
)


In [9]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose = 1, save_best_only=True)
model.fit(x_train,
          y_train,
          batch_size=32,
          epochs=4,
          validation_data=(x_valid, y_valid),
          callbacks=[checkpointer])

Epoch 1/4
150/150 [==============================] - 68s 455ms/step - loss: 12.4935 - root_mean_squared_error: 12.6798 - val_loss: 11.4350 - val_root_mean_squared_error: 11.5600

Epoch 00001: val_loss improved from inf to 11.43497, saving model to model.weights.best.hdf5
Epoch 2/4
150/150 [==============================] - 67s 448ms/step - loss: 11.0693 - root_mean_squared_error: 11.2123 - val_loss: 11.7647 - val_root_mean_squared_error: 11.8500

Epoch 00002: val_loss did not improve from 11.43497
Epoch 3/4
150/150 [==============================] - 69s 460ms/step - loss: 10.1066 - root_mean_squared_error: 10.2391 - val_loss: 12.1616 - val_root_mean_squared_error: 12.2589

Epoch 00003: val_loss did not improve from 11.43497
Epoch 4/4
150/150 [==============================] - 69s 460ms/step - loss: 9.3804 - root_mean_squared_error: 9.5232 - val_loss: 9.9623 - val_root_mean_squared_error: 10.0692

Epoch 00004: val_loss improved from 11.43497 to 9.96234, saving model to model.weights.bes

In [10]:
# Load the weights with the best validation loss
model.load_weights('model.weights.best.hdf5')
# Evaluate the model on test set
score = model.evaluate(x_test, y_test, verbose=0)
# Print test accuracy
print('\n', 'Test loss:', score)
y_hat = model.predict(x_test)
print(y_hat[455:996])
print(y_test[0])


 Test loss: [10.115452766418457, 10.245870590209961]
[[12.571361 ]
 [36.630756 ]
 [36.861008 ]
 [30.644924 ]
 [50.239582 ]
 [31.403784 ]
 [42.680885 ]
 [35.082523 ]
 [33.206432 ]
 [31.123983 ]
 [38.671642 ]
 [56.602592 ]
 [26.238617 ]
 [39.051807 ]
 [41.82472  ]
 [31.295727 ]
 [43.323532 ]
 [38.500454 ]
 [19.73041  ]
 [29.851091 ]
 [28.66594  ]
 [36.231373 ]
 [44.779114 ]
 [22.220325 ]
 [31.40183  ]
 [64.82799  ]
 [35.401035 ]
 [52.82444  ]
 [37.791996 ]
 [24.00309  ]
 [33.003952 ]
 [34.21906  ]
 [54.359104 ]
 [41.13431  ]
 [44.847977 ]
 [36.6339   ]
 [47.015568 ]
 [26.930431 ]
 [51.835827 ]
 [ 3.9026234]
 [42.98929  ]
 [40.42121  ]
 [60.992893 ]
 [26.775938 ]
 [31.860659 ]
 [50.60662  ]
 [52.916054 ]
 [36.438885 ]
 [40.667973 ]
 [25.155884 ]
 [34.672707 ]
 [29.20359  ]
 [33.668728 ]
 [39.648636 ]
 [39.955223 ]
 [32.656525 ]
 [19.193125 ]
 [54.42529  ]
 [35.62291  ]
 [30.869535 ]
 [25.82767  ]
 [31.310745 ]
 [39.64474  ]
 [53.954605 ]
 [56.72713  ]
 [42.124683 ]
 [43.34371  ]
 [25.756